# Data Cleansing

這個notebook主要說明下載完畢的資料要做什麼樣的基本處理才可以進入EDA的流程  
先前下載的資料多數為xml file，基本的結構不適合分析師直接操作  
這邊會說明如何透過封裝好的function來:  
* 解壓縮文件(.tar, .xml.gz)
* 資料結構化
* 資料儲存(這邊採用sqlite)  

# Lib

In [1]:
import pandas as pd
from tqdm import tqdm
import os
import time
import sys

# 將 hwttp 資料夾加入 Python 搜尋路徑
sys.path.append(os.path.abspath('..'))
import hwttp.data_scraper as ds

In [2]:
import hwttp.data_cleaning as dc

# Unzip data

注意解壓縮的時間範圍，因為原始檔案大小會佔據一定空間，貿然解壓縮可能會導致硬碟滿載  
建議第一次先解壓縮五天就好，再視自己的空間許可進行變更  

In [3]:
date_list = []
start_date = '20231101'
end_date = '20231101'
date_list = ds.generate_date_strings(start_date, end_date)

## VD data

這邊在處理上沒有特別區分靜態跟動態的VD資料，只要在對應的資料夾底下就可以一起解壓縮

In [25]:
dc.decompress_vd_data(date_list)

100%|██████████| 1/1 [00:23<00:00, 23.42s/it]


## ETag Info Data

1個月資料大小大概5GB，需要用的時候再解壓縮

In [26]:
dc.decompress_etag_info_data(date_list)

100%|██████████| 1/1 [00:01<00:00,  1.94s/it]


## ETag vol Data

In [27]:
dc.decompress_etag_vol_data(date_list)

100%|██████████| 1/1 [00:00<00:00,  3.51it/s]


## ETag Speed Data

In [28]:
dc.decompress_etag_speed_data(date_list)

100%|██████████| 1/1 [00:00<00:00,  2.51it/s]


## ETag Travel Time Data

In [37]:
dc.decompress_etag_intergantry_traveltime_data(date_list)

100%|██████████| 30/30 [00:00<00:00, 43.03it/s]


# Data ETL & store into sqlite DB

這部分會將xml結構的資料先轉換成dict，每種資料源的dict結構又會不太一樣，因此會使用獨立的function來轉換成dataframe  
這邊的目標是將dataframe全數儲存在db中，日後視需求再用sql撈取資料進行分析  
但需要注意的是儲存空間是否充足，因為資料的顆粒度很細，以sqlite儲存還是有一定負擔  
以下每個區塊會先附上原始資料平台所提供的schema做為參考  

資料的操作與儲存上在以下依據量體會分成兩種處理方式:  
* 所有資料儲存在同一張表: VD Static, VD Dynamic, ETag Static, ETag Pair, ETag Pair Live
    * 處理的腳本相對簡單   
* 資料依據時間維度分拆不同張表: ETag Vol, ETag Speed, ETag Travel Time
    * 處理的腳本相對複雜，額外於組裝多個條件邏輯進行判斷  

## VD Static

一天資料筆數 3,878筆  

In [1]:
# UpdateTime：	本平台資料更新時間(ISO8601格式:yyyy-MM-ddTHH:mm:sszzzz)
# UpdateInterval：	本平台資料更新週期(秒)
# AuthorityCode：	業管機關簡碼
# VDID：	VD設備代碼
# SubAuthorityCode：	業管子機關簡碼
# BiDirectional：	是否為雙向偵測 = ['0: 偵測單向', '1: 偵測雙向']
# LinkID：	基礎路段代碼, 請參閱[基礎路段代碼表]https://traffic-api-documentation.gitbook.io/traffic/xiang-dai-zhao-biao
# Bearing：	基礎路段方位 = ['N: 北', 'E: 東', 'W: 西', 'S: 南', 'NE: 東北', 'SE: 東南', 'NW: 西北', 'SW: 西南']
# RoadDirection：	基礎路段所屬道路方向, 請參閱[道路方向資料表]https://traffic-api-documentation.gitbook.io/traffic/xiang-dai-zhao-biao
# LaneNum：	設備於該方向基礎路段所能偵側車道數
# ActualLaneNum：	該方向基礎路段的實際車道數
# VDType：	VD 類別 = ['1: 線圈式', '2: 微波式', '3: 影像式', '4: 紅外線', '5: 超音波', '6: 其它']
# LocationType：	設置地點位置類型 = ['1: 路側', '2: 道路中央分隔島', '3: 快慢分隔島', '4: 車道上門架', '5: 車道鋪面', '6: 其他']
# DetectionType：	車流偵側類型 = ['1: 高快速公路/市快主線', '2: 高快速公路/市快匝道', '3: 其它道路路段中', '4: 其它道路路口(靠近路口或停止線)']
# PositionLon：	設備架設位置 X 坐標
# PositionLat：	設備架設位置 Y 坐標
# RoadID：	道路代碼, 請參閱[路名碼基本資料]https://traffic-api-documentation.gitbook.io/traffic/xiang-dai-zhao-biao
# RoadName：	道路名稱
# RoadClass：	道路分類 = ['0: 國道', '1: 快速道路', '2: 市區快速道路', '3: 省道', '4: 縣道', '5: 鄉道', '6: 市區一般道路', '7: 匝道']
# Start：	路段起點描述
# End：	路段迄點描述
# LocationMile：	所在方向里程數

這邊會先實體化一個db，需要指定table_name以及他的欄位名稱(依照順序)  
後續就是建立table，以及call function完成指定資料的清洗、轉換、儲存  

In [4]:
db_path = "../data/hwdb.db"
table_name = "VD_STATIC"
columns_in_order = '''
UpdateTime, UpdateInterval, AuthorityCode, VDID, SubAuthorityCode, 
BiDirectional, LinkID, Bearing, RoadDirection, Lane,
ActualLaneNum, VDType, LocationType, DetectionType, PositionLon,
PositionLat, RoadID, RoadName, RoadClass, Start, 
End, LocationMile
'''

vd_static_tb = dc.DatabaseManager(db_path=db_path, 
                                  table_name=table_name)

In [5]:
# 建立table
vd_static_tb.initialize_table(columns_in_order)

In [54]:
# Usage 實際需要塞資料時，請參考這邊，使用date_list去找到已經解壓縮路徑底下的資料
# 資料會需要先從xml解析成dict，過程中已經根據原始結構自訂好處理方式了
# 從dict轉換成df的時候會做一些特殊的處理，因為不同資料的字典結構會不太一樣

for date in tqdm(date_list):
    # VD_STATIC的資料都是以 "VD_0000.xml" 儲存
    xml_file_path = f'../data/raw/unzip_VD/{date}/VD_0000.xml'
    vd_0000 = dc.convert_xml_to_dict(xml_file_path)
    df = dc.vd_static_dict_to_df(vd_0000)
    vd_static_tb.append_data(df) # 可替換成update_data處理

100%|██████████| 1/1 [00:00<00:00,  3.32it/s]


In [2]:
# 如果需要刪除整張table的話
# vd_static_tb.delete_table_data()
# 如果怕資料有重複倒入，可以用update_data的方式處理，但會慢一些，因為要比對key value
# vd_static_tb.update_data(df, ['UpdateTime', 'LinkID'])

## VD Dynamic

一天的資料比數大概是 25236*1441 = 36,365,076 筆

In [3]:
# UpdateTime：	本平台資料更新時間(ISO8601格式:yyyy-MM-ddTHH:mm:sszzzz)
# UpdateInterval：	本平台資料更新週期(秒)
# AuthorityCode：	業管機關簡碼
# VDID：	VD設備代碼
# LinkID：	基礎路段代碼, 請參閱[基礎路段代碼表]https://traffic-api-documentation.gitbook.io/traffic/xiang-dai-zhao-biao
# LaneID：	車道代碼
# LaneType：	車道種類 = ['1: 一般車道', '2: 快車道', '3: 慢車道', '4: 機車道', '5: 高承載車道', '6: 公車專用道', '7: 轉向車道', '8: 路肩', '9: 輔助車道', '10: 調撥車道', '11: 其他']
# Speed：	平均速率偵測值, ['-99=資料異常', '-1=道路封閉']
# Occupancy：	佔有率偵測值
# VehicleType：	車種代碼, 請參閱[車種代碼]https://traffic-api-documentation.gitbook.io/traffic/xiang-dai-zhao-biao
# Volume：	流量偵測值, -99=資料異常
# speed(2)：	平均速率偵測值, -99=資料異常
# Status：	設備狀態 = ['0: 正常', '1: 通訊異常', '2: 停用或施工中', '3: 設備故障']
# DataCollectTime：	資料蒐集時間(ISO8601格式:yyyy-MM-ddTHH:mm:sszzzz)

In [6]:
db_path = "../data/hwdb.db"
table_name = "VD_DYNAMIC"
columns_in_order = '''
UpdateTime, UpdateInterval, AuthorityCode, VDID, LinkID, 
LaneID, LaneType, Speed, Occupancy, VehicleType,
Volume, Speed2, Status, DataCollectTime
'''

vd_dynamic_tb = dc.DatabaseManager(db_path=db_path, 
                                  table_name=table_name)

In [7]:
# 建立table
vd_dynamic_tb.initialize_table(columns_in_order)

In [8]:
date_list

['20231101']

In [9]:
# Usage 實際需要塞資料時，請參考這邊，使用date_list去找到已經解壓縮路徑底下的資料
# 資料會需要先從xml解析成dict，過程中已經根據原始結構自訂好處理方式了
# 從dict轉換成df的時候會做一些特殊的處理，因為不同資料的字典結構會不太一樣

for date in tqdm(date_list):
    # VD_DYNAMIC 的資料都是以 "VDLive_XXXX.xml" 儲存
    # xml_file_path = f'../data/raw/unzip_VD/{date}/VD_0000.xml'
    upper_file_path = f'../data/raw/unzip_VD/{date}/'
    file_name_list = os.listdir(upper_file_path)
    file_name_list = [file for file in file_name_list if file.startswith('VDLive')]

    for file in tqdm(file_name_list):
        xml_file_path = upper_file_path + file
        vdlive_0000 = dc.convert_xml_to_dict(xml_file_path)
        df = dc.vd_dynamic_dict_to_df(vdlive_0000)
        vd_dynamic_tb.append_data(df) # 可替換成update_data處理

  0%|          | 0/1 [00:00<?, ?it/s]

../data/raw/unzip_VD/20231101/VDLive_0000.xml
../data/raw/unzip_VD/20231101/VDLive_0001.xml
../data/raw/unzip_VD/20231101/VDLive_0002.xml
../data/raw/unzip_VD/20231101/VDLive_0003.xml
../data/raw/unzip_VD/20231101/VDLive_0004.xml
../data/raw/unzip_VD/20231101/VDLive_0005.xml
../data/raw/unzip_VD/20231101/VDLive_0006.xml
../data/raw/unzip_VD/20231101/VDLive_0007.xml
../data/raw/unzip_VD/20231101/VDLive_0008.xml
../data/raw/unzip_VD/20231101/VDLive_0009.xml
../data/raw/unzip_VD/20231101/VDLive_0010.xml
../data/raw/unzip_VD/20231101/VDLive_0011.xml
../data/raw/unzip_VD/20231101/VDLive_0012.xml
../data/raw/unzip_VD/20231101/VDLive_0013.xml
../data/raw/unzip_VD/20231101/VDLive_0014.xml
../data/raw/unzip_VD/20231101/VDLive_0015.xml
../data/raw/unzip_VD/20231101/VDLive_0016.xml
../data/raw/unzip_VD/20231101/VDLive_0017.xml
../data/raw/unzip_VD/20231101/VDLive_0018.xml
../data/raw/unzip_VD/20231101/VDLive_0019.xml
../data/raw/unzip_VD/20231101/VDLive_0020.xml
../data/raw/unzip_VD/20231101/VDLi

100%|██████████| 1/1 [23:10<00:00, 1390.76s/it]


In [ ]:
# 如果需要刪除整張table的話
# vd_dynamic_tb.delete_table_data()
# 如果怕資料有重複倒入，可以用update_data的方式處理，但會慢一些，因為要比對key value
# vd_dynamic_tb.update_data(df, ...)

## ETag Static

1天資料 339筆左右

In [4]:
# UpdateTime：	本平台資料更新時間(ISO8601格式:yyyy-MM-ddTHH:mm:sszzzz)
# UpdateInterval：	本平台資料更新週期(秒)
# AuthorityCode：	業管機關簡碼
# ETagGantryID：	eTag偵測站代碼
# LinkID：	基礎路段代碼, 請參閱[基礎路段代碼表]https://traffic-api-documentation.gitbook.io/traffic/xiang-dai-zhao-biao
# LocationType：	設置地點位置類型 = ['1: 路側', '2: 道路中央分隔島', '3: 快慢分隔島', '4: 車道上門架', '5: 車道鋪面', '6: 其他']
# PositionLon：	設備架設位置 X 坐標
# PositionLat：	設備架設位置 Y 坐標
# RoadID：	道路代碼, 請參閱[路名碼基本資料]https://traffic-api-documentation.gitbook.io/traffic/xiang-dai-zhao-biao
# RoadName：	道路名稱
# RoadClass：	道路分類 = ['0: 國道', '1: 快速道路', '2: 市區快速道路', '3: 省道', '4: 縣道', '5: 鄉道', '6: 市區一般道路', '7: 匝道']
# RoadDirection：	基礎路段所屬道路方向, 請參閱[道路方向資料表]https://traffic-api-documentation.gitbook.io/traffic/xiang-dai-zhao-biao
# Start：	路段起點描述
# End：	路段迄點描述
# LocationMile：	所在方向里程數

In [10]:
db_path = "../data/hwdb.db"
table_name = "ETAG_STATIC"
columns_in_order = '''
UpdateTime, UpdateInterval, AuthorityCode, LinkVersion, 
ETagGantryID, LinkID, LocationType, PositionLon, PositionLat, 
RoadID, RoadName, RoadClass, RoadDirection, Start, End, LocationMile
'''

etag_static_tb = dc.DatabaseManager(db_path=db_path, 
                                  table_name=table_name)

In [11]:
# 建立table
etag_static_tb.initialize_table(columns_in_order)

In [12]:
# Usage 實際需要塞資料時，請參考這邊，使用date_list去找到已經解壓縮路徑底下的資料
# 資料會需要先從xml解析成dict，過程中已經根據原始結構自訂好處理方式了
# 從dict轉換成df的時候會做一些特殊的處理，因為不同資料的字典結構會不太一樣

for date in tqdm(date_list):
    # ETAG_STATIC 的資料都是以 "ETag_0000.xml" 儲存
    upper_file_path = f'../data/raw/unzip_ETag/{date}/'
    file_name_list = os.listdir(upper_file_path)
    file_name_list = [file for file in file_name_list if file.startswith('ETag_0000')]

    for file in file_name_list:
        xml_file_path = upper_file_path + file
        etag_0000 = dc.convert_xml_to_dict(xml_file_path)
        df = dc.etag_static_dict_to_df(etag_0000)
        etag_static_tb.append_data(df) # 可替換成update_data處理

100%|██████████| 1/1 [00:00<00:00, 25.71it/s]


In [5]:
# 如果需要刪除整張table的話
# etag_static_tb.delete_table_data()
# 如果怕資料有重複倒入，可以用update_data的方式處理，但會慢一些，因為要比對key value
# etag_static_tb.update_data(df, ...)

## ETag Pair

一天大概327筆，路徑關係理論上會是固定的，除非有變化

In [6]:
# UpdateTime：	本平台資料更新時間(ISO8601格式:yyyy-MM-ddTHH:mm:sszzzz)
# UpdateInterval：	本平台資料更新週期(秒)
# AuthorityCode：	業管機關簡碼
# ETagPairID：	eTag 配對路徑代碼
# StartETagGantryID：	eTag配對起始點偵測站代碼
# EndETagGantryID：	eTag配對結束點偵測站代碼
# Description：	配對路徑文字描述
# Distance：	配對路徑距離, GIS提供的配對路徑距離(KM), 可到小數點3位
# StartLinkID：	起點基礎路段代碼, 請參閱[基礎路段代碼表]https://traffic-api-documentation.gitbook.io/traffic/xiang-dai-zhao-biao
# EndLinkID：	迄點基礎路段代碼, 請參閱[基礎路段代碼表]https://traffic-api-documentation.gitbook.io/traffic/xiang-dai-zhao-biao
# Geometry：	配對路徑線型圖資資料, 格式為WKT

In [13]:
db_path = "../data/hwdb.db"
table_name = "ETAG_PAIR"
columns_in_order = '''
UpdateTime, UpdateInterval, AuthorityCode, ETagPairID,
StartETagGantryID, EndETagGantryID, Description, Distance,
StartLinkID, EndLinkID, Geometry
'''

etag_pair_tb = dc.DatabaseManager(db_path=db_path, 
                                  table_name=table_name)

In [14]:
# 建立table
etag_pair_tb.initialize_table(columns_in_order)

In [17]:
# Usage 實際需要塞資料時，請參考這邊，使用date_list去找到已經解壓縮路徑底下的資料
# 資料會需要先從xml解析成dict，過程中已經根據原始結構自訂好處理方式了
# 從dict轉換成df的時候會做一些特殊的處理，因為不同資料的字典結構會不太一樣

for date in tqdm(date_list):
    # ETAG_STATIC 的資料都是以 "ETagPair_0000.xml" 儲存
    upper_file_path = f'../data/raw/unzip_ETag/{date}/'
    file_name_list = os.listdir(upper_file_path)
    file_name_list = [file for file in file_name_list if file.startswith('ETagPair_0000')]

    for file in file_name_list:
        xml_file_path = upper_file_path + file
        etagpair_0000 = dc.convert_xml_to_dict(xml_file_path)
        df = dc.etagpair_dict_to_df(etagpair_0000)
        etag_pair_tb.append_data(df) # 可替換成update_data處理

100%|██████████| 1/1 [00:00<00:00, 35.81it/s]


In [7]:
# 如果需要刪除整張table的話
# etag_pair_tb.delete_table_data()
# 如果怕資料有重複倒入，可以用update_data的方式處理，但會慢一些，因為要比對key value
# etag_pair_tb.update_data(df, ...)

## ETag Pair Live

1天大概 606,240筆

In [8]:
# UpdateTime：	本平台資料更新時間(ISO8601格式:yyyy-MM-ddTHH:mm:sszzzz)
# UpdateInterval：	本平台資料更新週期(秒)
# AuthorityCode：	業管機關簡碼
# ETagPairID：	ETag 配對路徑編號
# StartETagStatus：	配對起始點設備狀態 = ['0: 正常', '1: 通訊異常', '2: 停用或施工中', '3: 設備故障']
# EndETagStatus：	配對結束點設備狀態 = ['0: 正常', '1: 通訊異常', '2: 停用或施工中', '3: 設備故障']
# VehicleType：	車種代碼, 請參閱[車種代碼]https://traffic-api-documentation.gitbook.io/traffic/xiang-dai-zhao-biao
# TravelTime：	平均旅行時間(指定車種下), 單位:秒, -99=資料異常
# StandardDeviation：	配對樣本數之旅行時間標準差, 單位:秒, -99=資料異常
# SpaceMeanSpeed：	平均車速 (指定車種下), 單位:KM/Hr, -99=資料異常
# VehicleCount：	配對樣本數(指定車種下), 單位:輛
# StartTime：	資料蒐集起始時間(指通過迄點資料) (ISO8601格式:yyyy-MM-ddTHH:mm:sszzzz)
# EndTime：	資料蒐集結束時間(指通過迄點資料) (ISO8601格式:yyyy-MM-ddTHH:mm:sszzzz)
# DataCollectTime：	資料蒐集時間(ISO8601格式:yyyy-MM-ddTHH:mm:sszzzz)

In [21]:
db_path = "../data/hwdb.db"
table_name = "ETAG_PAIR_LIVE"
columns_in_order = '''
UpdateTime, UpdateInterval, AuthorityCode, ETagPairID,
StartETagStatus, EndETagStatus, VehicleType, TravelTime,
StandardDeviation, SpaceMeanSpeed, VehicleCount, StartTime,
EndTime, DataCollectTime
'''

etag_pairlive_tb = dc.DatabaseManager(db_path=db_path, 
                                  table_name=table_name)

In [22]:
# 建立table
etag_pairlive_tb.initialize_table(columns_in_order)

In [30]:
# Usage 實際需要塞資料時，請參考這邊，使用date_list去找到已經解壓縮路徑底下的資料
# 資料會需要先從xml解析成dict，過程中已經根據原始結構自訂好處理方式了
# 從dict轉換成df的時候會做一些特殊的處理，因為不同資料的字典結構會不太一樣
for date in tqdm(date_list):
    # ETAG_STATIC 的資料都是以 "ETagPairLive_XXXX.xml" 儲存
    upper_file_path = f'../data/raw/unzip_ETag/{date}/'
    file_name_list = os.listdir(upper_file_path)
    file_name_list = [file for file in file_name_list if file.startswith('ETagPairLive_')]

    for file in file_name_list:
        xml_file_path = upper_file_path + file
        etagpairlive_ = dc.convert_xml_to_dict(xml_file_path)
        try:
            df = dc.etagpairlive_dict_to_df(etagpairlive_)
            etag_pairlive_tb.append_data(df) # 可替換成update_data處理
        except:
            print(f'{date}資料毀損，檔案為 {file}，先跳過')

100%|██████████| 1/1 [00:27<00:00, 27.83s/it]


In [9]:
# 如果需要刪除整張table的話
# etag_pairlive_tb.delete_table_data()
# 如果怕資料有重複倒入，可以用update_data的方式處理，但會慢一些，因為要比對key value
# etag_pairlive_tb.update_data(df, ...)

## ETag Vol

原始資料為 csv file，每天每小時每五分鐘為一份文件，因此集結起來數量相當大  
再儲存於輕量級DB中最好不要全年塞再同一張表裡，因此以每個月一張table放置會較合適  
每個月一張的話，大概是15,132,960筆    

In [10]:
# TimeStamp：	報表時間戳記，即時資料與更新資料時間戳記代表的時間區間不同
#             即時資料：2024-01-01 10:00代表時間區間為2024-01-01 09:55~10:00
#             更新資料：2024-01-01 10:00代表時間區間為2024-01-01 10:00~10:05
# GantryID：	門架編號，門架所在路段詳使用手冊
# Direction：	車行方向，S南、N北
# VehicleType：	車種，31小客車、32小貨車、41大客車、42大貨車、5聯結車
# Volume：	交通量，時間區間內經過單一門架之車流量總量

因為這邊開始要用一個月唯一張表的方式來儲存，需要call一個function來解決

In [7]:
daterange_combinations = dc.generate_daterange_combinations(2023, 11, 2023, 11)
print(daterange_combinations)

[['20231101', '20231130']]


In [13]:
# predefine，後續主要修改的就是這塊而已
upper_file_path = '../data/raw/unzip_etag_gantry_vol/M03A'
db_path = "../data/hwdb.db"
table_name_prefix = "ETAG_M03A"
columns_in_order = '''
TimeStamp, GantryID, Direction, VehicleType, Volume
'''
col_names = columns_in_order.replace(' ', '').replace('\n', '').split(',')

# 組裝
for index, item in enumerate(daterange_combinations):
    start_date, end_date = item[0], item[1]
    current_year_month = start_date[0:6]
    date_list = ds.generate_date_strings(start_date, end_date)
    
    # define db table name and instance the db manager
    table_name = f"{table_name_prefix}_{current_year_month}"
    etag_temp_tb = dc.DatabaseManager(db_path=db_path, table_name=table_name)
    
    # initialize table
    try: 
        etag_temp_tb.initialize_table(columns_in_order)
        print(f'table: {table_name} initialized')
    except:
        print('something wrong with the initialization')
        
    # append table
    try:
        # 每小時會是一組資料夾
        time_list = [f'{i:02}' for i in range(24)]
        for date in tqdm(date_list):
            # 鎖定特定日期資料夾，只拿包含VDLive開頭的資料名稱
            for time in time_list: 
                file_name_list = os.listdir(f'{upper_file_path}/{date}/{time}/')
                file_name_list = [file for file in file_name_list if file.startswith('TDCS_M03A_')]
            
                for file in file_name_list:
                    csv_file_path = f'{upper_file_path}/{date}/{time}/{file}'
                    df = pd.read_csv(csv_file_path, names=col_names)
                    df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], format='%Y-%m-%d %H:%M')
                    etag_temp_tb.append_data(df)
    except:
        print(f'something wrong during the table append procedure under {current_year_month}')
    
    print(f'{table_name} data insertion complete!')

table: ETAG_M03A_202311 initialized


  3%|▎         | 1/30 [00:05<02:51,  5.91s/it]

something wrong during the table append procedure under 202311
ETAG_M03A_202311 data insertion complete!


上面展示的部分因為只有放入一天資料(488,160筆)，所以會進入except環節  

## ETag Speed

1天的資料大概是 606,240筆

In [11]:
# TimeStamp：	報表時間戳記，即時資料與更新資料時間戳記代表的時間區間不同
#             即時資料：2024-01-01 10:00代表時間區間為2024-01-01 09:55~10:00
#             更新資料：2024-01-01 10:00代表時間區間為2024-01-01 10:00~10:05
# GantryFrom：	上游門架編號，門架所在路段詳使用手冊
# GantryTo：	下游門架編號，門架所在路段詳使用手冊
# VehicleType：	車種，31小客車、32小貨車、41大客車、42大貨車、5聯結車
# Speed：	中位數車速，計算單位為秒
# Volume：	交通量，時間區間內經過上下游門架之車流量總量

要特別注意的是塞入sqlite db 中TimeStamp的格式要先處理好，Vol, Speed, Travel Time在裡面稍微不太一樣，需要異動  

In [34]:
# predefine，後續主要修改的就是這塊而已
upper_file_path = '../data/raw/unzip_etag_intergantry_speed/M05A'
db_path = "../data/hwdb.db"
table_name_prefix = "ETAG_M05A"
columns_in_order = '''
TimeStamp, GantryFrom, GantryTo, VehicleType, Speed, Volume
'''
col_names = columns_in_order.replace(' ', '').replace('\n', '').split(',')

# 組裝
for index, item in enumerate(daterange_combinations):
    start_date, end_date = item[0], item[1]
    current_year_month = start_date[0:6]
    date_list = ds.generate_date_strings(start_date, end_date)
    
    # define db table name and instance the db manager
    table_name = f"{table_name_prefix}_{current_year_month}"
    etag_temp_tb = dc.DatabaseManager(db_path=db_path, table_name=table_name)
    
    # initialize table
    try: 
        etag_temp_tb.initialize_table(columns_in_order)
        print(f'table: {table_name} initialized')
    except:
        print('something wrong with the initialization')
        
    # append table
    try:
        # 每小時會是一組資料夾
        time_list = [f'{i:02}' for i in range(24)]
        for date in tqdm(date_list):
            # 鎖定特定日期資料夾，只拿包含VDLive開頭的資料名稱
            for time in time_list: 
                file_name_list = os.listdir(f'{upper_file_path}/{date}/{time}/')
                file_name_list = [file for file in file_name_list if file.startswith('TDCS_M05A_')]
                for file in file_name_list:
                    csv_file_path = f'{upper_file_path}/{date}/{time}/{file}'
                    df = pd.read_csv(csv_file_path, names=col_names)
                    df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], format='%Y/%m/%d %H:%M')
                    etag_temp_tb.append_data(df)
    except:
        print(f'something wrong during the table append procedure under {current_year_month}')
    
    print(f'{table_name} data insertion complete!')

table: ETAG_M05A_202311 initialized


  3%|▎         | 1/30 [00:04<02:07,  4.38s/it]

something wrong during the table append procedure under 202311
ETAG_M05A_202311 data insertion complete!


## ETag Travel Time

1天大概 606,240筆

TimeStamp：	報表時間戳記，以下游門架時間戳記代表，此外即時與更新資料時間戳記代表的時間區間不同  
            即時資料：2024-01-01 10:00代表時間區間為2024-01-01 09:55 ~ 10:00  
            更新資料：2024-01-01 10:00代表時間區間為2024-01-01 10:00 ~ 10:05  
GantryFrom：	上游門架編號，門架所在路段詳使用手冊  
GantryTo：	下游門架編號，門架所在路段詳使用手冊  
VehicleType：	車種，31小客車、32小貨車、41大客車、42大貨車、5聯結車  
TravelTime：	中位數旅行時間，計算單位為秒  
Traffic：	交通量，時間區間內經過上下游門架之車流量總量  

In [46]:
# predefine，後續主要修改的就是這塊而已
upper_file_path = '../data/raw/unzip_etag_intergantry_traveltime/M04A'
db_path = "../data/hwdb.db"
table_name_prefix = "ETAG_M04A"
columns_in_order = '''
TimeStamp, GantryFrom, GantryTo, VehicleType, TravelTime, Traffic
'''
col_names = columns_in_order.replace(' ', '').replace('\n', '').split(',')

# 組裝
for index, item in enumerate(daterange_combinations):
    start_date, end_date = item[0], item[1]
    current_year_month = start_date[0:6]
    date_list = ds.generate_date_strings(start_date, end_date)
    
    # define db table name and instance the db manager
    table_name = f"{table_name_prefix}_{current_year_month}"
    etag_temp_tb = dc.DatabaseManager(db_path=db_path, table_name=table_name)
    
    # initialize table
    try: 
        etag_temp_tb.initialize_table(columns_in_order)
        print(f'table: {table_name} initialized')
    except:
        print('something wrong with the initialization')
        
    # append table
    try:
        # 每小時會是一組資料夾
        time_list = [f'{i:02}' for i in range(24)]
        for date in tqdm(date_list):
            # 鎖定特定日期資料夾，只拿包含VDLive開頭的資料名稱
            for time in time_list: 
                file_name_list = os.listdir(f'{upper_file_path}/{date}/{time}/')
                file_name_list = [file for file in file_name_list if file.startswith('TDCS_M04A_')]
                for file in file_name_list:
                    csv_file_path = f'{upper_file_path}/{date}/{time}/{file}'
                    df = pd.read_csv(csv_file_path, names=col_names)
                    df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], format='%Y/%m/%d %H:%M')
                    etag_temp_tb.append_data(df)
    except:
        print(f'something wrong during the table append procedure under {current_year_month}')
    
    print(f'{table_name} data insertion complete!')

table: ETAG_M04A_202311 initialized


  7%|▋         | 2/30 [00:08<02:05,  4.47s/it]

something wrong during the table append procedure under 202311
ETAG_M04A_202311 data insertion complete!
